In [ ]:
import jsonrpcclient
import subprocess
from os.path import join
from chainrunner import Chain, Seq, generate_keys, check_finalized

In [ ]:
# Path to working directory, where chainspec, logs and nodes' dbs are written:
workdir = '/tmp/workdir'
# Path to the pre-update aleph-node binary:
oldbin = join(workdir,'aleph-node-old')
# Path to the post-update aleph-node binary:
newbin = join(workdir,'aleph-node-new')
# Path to the post-update compiled runtime:
runtime = join(workdir,'aleph_runtime.compact.wasm')
# Path to cliain:
cliain = '../bin/cliain/target/release/cliain'

In [ ]:
def query_runtime_version(nodes):
    print('Current version:')
    versions = set()
    for i, node in enumerate(nodes):
        sysver = node.rpc('system_version').result
        resp = node.rpc('state_getRuntimeVersion')
        if isinstance(resp, jsonrpcclient.Ok):
            rt = resp.result['specVersion']
            versions.add(rt)
        else:
            rt = "ERROR"
        print(f'  Node {i}: system: {sysver}  runtime: {rt}')
    if len(versions) > 1:
        print(f'ERROR: nodes reported different runtime versions: {versions}')
    if versions:
        return max(versions)
    return -1

In [ ]:
phrases = ['//Cartman','//Stan','//Kyle','//Kenny']
keys = generate_keys(newbin, phrases)

In [ ]:
chain = Chain(workdir)
chain.bootstrap(oldbin,
                keys.values(),
                sudo_account_id=keys[phrases[0]],
                chain_type='local')

In [ ]:
chain.set_flags('validator',
                port=Seq(30334),
                ws_port=Seq(9944),
                rpc_port=Seq(9933),
                unit_creation_delay=200,
                execution='Native')
addresses = [n.address() for n in chain]
chain.set_flags(public_addr=addresses)

In [ ]:
chain.start('old')

In [ ]:
check_finalized(chain)
query_runtime_version(chain)

In [ ]:
chain[3].stop()
chain[3].purge()
chain[3].binary = newbin
chain[3].start('new3')

In [ ]:
check_finalized(chain)
oldver = query_runtime_version(chain)

In [ ]:
subprocess.check_call(
    [cliain, '--node', 'localhost:9945', '--seed', phrases[0],
        'update-runtime', '--runtime', runtime],
    env=dict(os.environ, RUST_LOG="warn"))

In [ ]:
check_finalized(chain)
newver = query_runtime_version(chain)

In [ ]:
chain.stop(nodes=[0,1,2])
chain.set_binary(newbin, nodes=[0,1,2])
chain.start('new', nodes=[0,1,2])

In [ ]:
check_finalized(chain)
query_runtime_version(chain)

In [ ]:
chain.stop()

In [ ]:
hf = min(node.highest_block()[1] for node in chain)
print(f'Sanity check: the highest finalized block is {hf}. Comparing exported states after that block:')
if chain[0].state(hf) == chain[1].state(hf) == chain[2].state(hf) == chain[3].state(hf):
    print("The same :)")
else:
    print("DIFFERENT!")

In [ ]:
if oldver == newver:
    print("ERROR: runtime version reported by nodes didn't change after the update")
